### Video Loading Tests

This file is for testing my methods for loading in the video data and manipulating it so that is ready for analysis/training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 # OpenCV
import os

import Settings # My global settings
import Preprocessing # Work with the raw data
import Sampling

In [2]:
Preprocessing.testData()

Searching for data in location specified by Settings.DATA_LOCATION:
/run/media/jack/Seagate Portable Drive/Research/geogran2/
...
Found 55 possible data files using listDataFiles()
Verified 55 properties files exist
Example of property file keys:
dict_keys(['name', 'velocity', 'shearspringname', 'normalspringname', 'numpart', 'boundary', 'starttime', 'goodtime', 'endtime'])

Verified 55 mat files exist
Example of mat file keys:
dict_keys(['t', 'x', 'f', 'fn', 'good', 'multipeak', 'time', 'position', 'force', 'duration', 'slip', 'deltaF', 'work'])

Found 26 data files with movies using listDataFiles(True)
Verified 26 movies exist


True

In [3]:
dataFiles = Preprocessing.listDataFiles(excludeDataWithoutMovies=True)
#matFile = Preprocessing.loadMatFile(dataFiles[0])
#video = Preprocessing.loadVideo([dataFiles[0], dataFiles[7], dataFiles[18]], ignoreInvalidData=True)

propertyFiles = Preprocessing.loadRunData(dataFiles)
videos = Preprocessing.loadVideo(dataFiles)
matFiles = Preprocessing.loadMatFile(dataFiles)
print(dataFiles[0])
print(Preprocessing.loadRawSensorData(dataFiles[0]))
#print(matFiles[0])
#print(matFile["t"][:,0])
#print(matFile)
#print(propertyFile)\

070330b
[[0.000000e+00 3.333090e-01 1.686410e-01]
 [1.000000e-02 3.318520e-01 1.686410e-01]
 [2.000000e-02 3.324990e-01 1.688030e-01]
 ...
 [6.589700e+02 1.251046e+00 2.885915e+00]
 [6.589800e+02 1.229997e+00 2.890125e+00]
 [6.589900e+02 1.246026e+00 2.903078e+00]]


In [5]:
print(Preprocessing.loadMatFile('070524b'))

{'t': array([0.0000e+00, 1.0000e-02, 2.0000e-02, ..., 6.9697e+02, 6.9698e+02,
       6.9699e+02]), 'x': array([ 12.1161921,  12.0945165,  12.126963 , ..., 187.0883253,
       187.1532852, 187.0341363]), 'f': array([ 4.50859,  4.51345,  4.47135, ..., 16.07585, 16.12118, 16.06937]), 'fn': array([0.183213, 0.183375, 0.183052, ..., 2.901945, 2.895144, 2.895306]), 'good': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 

In [4]:
print(len(matFiles[0]["good"]))
#print(propertyFiles[0])
#print(matFiles[0])

451
{'name': '070330b', 'velocity': 0.05, 'shearspringname': 'D', 'normalspringname': '#1', 'boundary': '', 'starttime': 15.0, 'endtime': 650.0, 'fps': 4.0, 'pixpermm': 1.8, 'type': "{'creep'}", 'goodarea': array([ 90, 697,  79, 493]), 'centerline': 270.0, 'numpart': 'N'}


In [3]:
Sampling.playVideo(videos[0])

Played 346 frames


In [31]:
keys = []
values = []
with open('070525d.rundata') as file:
    for line in file.read().split('\n'):
        try:
            newKey, newValue = line.split('=')
            keys.append(newKey.strip())
            values.append(newValue.strip())
        except:
            pass # Ignore whitespace lines
    
runProperties = dict(zip(keys, values))

# Convert the good area to be a proper array
runProperties["goodarea"] = np.array([i.strip() for i in runProperties["goodarea"][1:-2].split()], dtype='int')

print(runProperties)

{'name': '070525d', 'velocity': '0.05', 'shearspringname': 'B', 'normalspringname': 'C2+2', 'numpart': 'N', 'boundary': '', 'fps': '4', 'pixpermm': '1.80', 'type': "{'GR'}", 'goodarea': array([ 79, 746, 100, 497]), 'centerline': '272', 'sync': '2.5', 'starttime': '5', 'goodtime': '120', 'endtime': '413', 'maskout': '[10:30]'}


In [48]:
video = cv2.VideoCapture('070525d.bw.avi')

if not video.isOpened():
    print('Error opening file')

    
ret, frame = video.read()
grayscaleFrame = frame[:,:,0]

reg = runProperties["goodarea"]

cv2.imshow('Frame', grayscaleFrame[reg[0]:reg[1], reg[2]:reg[3]])
cv2.imshow('Frame', grayscaleFrame)
cv2.waitKey(50000)

# while video.isOpened():
#     ret, frame = video.read()

#     if ret:
#         cv2.imshow('Frame', frame)
#         cv2.waitKey(25)
        
#     else:
#         break
        
video.release()

cv2.destroyAllWindows()